### Variables are memory references
---

**Python Memory Manager** gets data from the **heap**
```python
v = 10
```
If ```10``` goes to the **heap**, then ```v``` is a  **reference** to a memory slot

In [1]:
v = 10
print(hex(id(v)))

0x7ff8df77a2b0


In [2]:
greeting = "Hello"
print(greeting)

Hello


In [3]:
print(hex(id(greeting)))

0x2b6657c4d70


### Reference count
---

In [4]:
import sys
import ctypes

In [5]:
a = [1, 2, 3]
print(hex(id(a)))

0x2b6657bd7c8


In [6]:
sys.getrefcount(a)

2

In [7]:
def ref_count(address: int):
  return ctypes.c_long.from_address(address).value

In [8]:
ref_count(id(a))

1

In [9]:
b = a
ref_count(id(a))

2

In [10]:
c = a
ref_count(id(a))

3

In [11]:
c = 3
b = 4
ref_count(id(a))

1

In [12]:
a = None
ref_count(id(a))

26994

In [13]:
for i in range(10):
  print(ref_count(id(a)))

26983
26985
26987
26989
26981
26983
26985
26987
26989
26991


### Garbage collection
---

Can be controlled programmatically using ```gc```

Mainly there to clean up **circular references**

In [14]:
# Circular reference

import gc

In [15]:
def object_by_id(object_id):
  for obj in gc.get_objects():
    if id(obj) == object_id:
      return "Objects exist"
  return "Not found"

In [16]:
class A:
  def __init__(self):
    self.b = B(self)  # Self is instance of A
    print(f"A: self={hex(id(self))}, b={hex(id(self.b))}")

In [17]:
class B:
  def __init__(self, a):
    self.a = a
    print(f"B: self={hex(id(self))}, a={hex(id(self.a))}")

In [18]:
gc.disable()

In [19]:
my_var = A()

B: self=0x2b6656d4408, a=0x2b6656d4988
A: self=0x2b6656d4988, b=0x2b6656d4408


In [20]:
print(hex(id(my_var)))
print(hex(id(my_var.b)))
print(hex(id(my_var.b.a)))

0x2b6656d4988
0x2b6656d4408
0x2b6656d4988


In [21]:
a_id = id(my_var)
b_id = id(my_var.b)

In [22]:
print(ref_count(a_id))
print(ref_count(b_id))
print(object_by_id(a_id))
print(object_by_id(b_id))

2
1
Objects exist
Objects exist


In [23]:
my_var = None
print(ref_count(a_id))
print(ref_count(b_id))
print(object_by_id(a_id))
print(object_by_id(b_id))

1
1
Objects exist
Objects exist


In [24]:
gc.collect()
print(ref_count(a_id))
print(ref_count(b_id))
print(object_by_id(a_id))
print(object_by_id(b_id))

0
0
Not found
Not found


### Dynamic vs static typing
---